In [1]:
# code adapted from https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33#:~:text=The%20Keras%20ResNet%20got%20to,to%20do%20with%20weight%20initializations.
# import plaidml
# import plaidml.keras
# plaidml.install_backend()
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
NUM_SAMP = 1000
import random
random.seed(10)

from PIL import Image
from glob import glob

# Clean Script
from glob import glob
import tensorflow as tf

from tensorflow.keras.utils import load_img

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model

# Collect paths to images based on label
nonCancerPaths = glob('imagedata/*/0/*.png')
cancerousPaths = glob('imagedata/*/1/*.png')

# Turn filepaths into image arrays to train a model
def paths_to_image(paths, label, num_samples):
    images = []
    labels = []
    for path in paths[0:num_samples]:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
    return [images[0: num_samples], labels[0: num_samples]]
nonCancerImages = paths_to_image(nonCancerPaths, 0, NUM_SAMP)
cancerImages = paths_to_image(cancerousPaths, 1, NUM_SAMP)







2023-03-19 14:02:07.328188: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
allImages = nonCancerImages[0] + cancerImages[0]
allLabels = nonCancerImages[1] + cancerImages[1]

x_train, x_test, y_train, y_test = train_test_split(allImages, allLabels, test_size=.20, stratify= allLabels, random_state = 10)

datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen.fit(x_train)

train_iterator = datagen.flow(np.array(x_train), np.array(y_train))
val_iterator = datagen.flow(np.array(x_test), np.array(y_test))
# x_train = tf.keras.applications.vgg16.preprocess_input(np.array(x_train))
# x_test = tf.keras.applications.vgg16.preprocess_input(np.array(x_test))

print(np.array(x_train).shape)
print(np.array(y_train).shape)




(1600, 50, 50, 3)
(1600,)


In [ ]:
from keras.optimizers import Adam

# #Learning rates above .01 are not useful

# learning_rates = [.0001, .0005, .001, .005, .01, .05, .1]

# for lr in learning_rates:
#     model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
#     adam = Adam(learning_rate=lr)
#     model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

#     print("FITTING")
#     history = model.fit(np.array(x_train), np.array(y_train), epochs=10, validation_data=(np.array(x_test), np.array(y_test)))
#     plt.plot(history.history['accuracy'], label='accuracy')
#     plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.ylim([0.5, 1])
#     plt.legend(loc='lower right')
#     plt.title("Learning Rate: " + str(lr))
#     plt.show()



# LR = .0001 achieves the highest validation accuracy
# Increase number of epochs to see if more epochs further increases val error
model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
adam = Adam(learning_rate=.0001)
model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

print("FITTING")
history = model.fit(train_iterator, epochs=15, validation_data=val_iterator)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.title("Learning Rate: " + str(lr))
plt.show()



FITTING
Epoch 1/15
50/50 [==============================] - 42s 819ms/step - loss: 0.6866 - accuracy: 0.5281 - val_loss: 0.6849 - val_accuracy: 0.5000
Epoch 2/15
50/50 [==============================] - 42s 830ms/step - loss: 0.5464 - accuracy: 0.7369 - val_loss: 0.4080 - val_accuracy: 0.8125
Epoch 3/15
11/50 [=====>........................] - ETA: 33s - loss: 0.4120 - accuracy: 0.8381

In [ ]:
# LR = .0001 achieves the highest validation accuracy
# Increase number of epochs to see if more epochs further increases val error
model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
adam = Adam(learning_rate=.0001)
model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

print("FITTING")
history = model.fit(np.array(x_train), np.array(y_train), epochs=15, validation_data=(np.array(x_test), np.array(y_test)))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.title("Learning Rate: " + str(lr))
plt.show()


In [ ]:
model.summary()

In [ ]:
def get_test_data(paths, start, stop, label):
    images = []
    labels = []
    for path in paths[start:stop]:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
        
    return [tf.keras.applications.vgg16.preprocess_input(np.array(images)), np.array(labels)]
        
cancer_test = get_test_data(cancerousPaths, 20000, 30000, 1)
benign_test = get_test_data(nonCancerPaths, 20000, 30000, 0)
model.evaluate(cancer_test[0], cancer_test[1])
model.evaluate(benign_test[0], benign_test[1])

In [ ]:
np.max(x_train)


In [ ]:
np.max(x_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(allImages, allLabels, test_size=.20, stratify= allLabels, random_state = 10)

In [ ]:
np.max(x_train)

In [ ]:
np.min(x_train)

In [ ]:
np.std(x_train)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
len(nonCancerPaths)

In [ ]:
len(cancerousPaths)